# Project Title

### Project Abstract

Write Description Here (why we chose this subject, the general things we seeked to find out, etc, idk)

Introduction: We are Team (team name, team members) and our project focused on analyzing (description of project).  Our project goals included:
* Goal 1
* Goal 2
* Etc.

Collaboration: We collaborated using the following tools:
(collaboration tool) for (purpose),
(collaboration tool) for (purpose), etc.

In [3]:
# SOMETHING IN THIS CELL IS TOO MUCH MEMORY TO IMPORT; LET'S FIND OUT WHAT
# PROBABLY MATPLOTLIB OR PLOTLY??? That's the only thing different. I moved both to the cell below

"""
If necessary, uncomment and install:
"""
!pip install wbdata
!pip install cufflinks


import wbdata
import cufflinks as cf
import pandas as pd
import numpy as np



In [ ]:
# leaving here because danger? run at ur own risk

import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
# Later we will not need these cells, because they're only for reference
# Delete now or later

SOURCE = 87
indicators = wbdata.get_indicator(source=SOURCE)
indicators

id                          name
--------------------------  -------------------------------------------------------------------------------------------------------------------------------------------------------------------------
account.t.d                 Account (% age 15+)
account.t.d.7               Account, income, poorest 40% (% ages 15+)
AG.LND.AGRI.ZS              Agricultural land (% of land area)
AG.LND.ARBL.ZS              Arable land (% of land area)
AG.LND.FRST.ZS              Forest area (% of land area)
CC.ADPO.MAEX.AA             Additional population exposed to annual coastal floods due to sea level rise, as a share of actual population (%) - Min. exposure, 2100
CC.ADPO.MAEX.BB             Additional population exposed to annual coastal floods due to sea level rise, as a share of actual population (%) - Max exposure, 2100
CC.ADPO.MIEX.AA             Additional population exposed to annual coastal floods due to sea level rise, as a share of actual population (%) - Min

In [ ]:
# Later we will not need these cells, because they're only for reference
# Delete now or later

SOURCE = 87 
# 40 - "Population estimates and projections"
# 87 - 'Country Climate and Development Report'

indicators = wbdata.get_indicator(source=SOURCE)
indicators

## Population Statistic Function

It's a long function, but it is incredibly specifc; you can estimate any combination of ages, including a single age. It can do much more than just capture the already-established bins. It creates corresponding ratios when an age lands somewhere in the middle of a bin.

In [7]:
def population(country, year, sex, lower_age, upper_age):
    '''Function returns a SINGLE computed statistic given all the variables.'''
    # 'sex' inputs as either 'All' (for 'All people'), or 'Male' or 'Female'
    
    def build_year_labelled_df(variable_labels, year, country):
        '''Creates a dataframe with 1 or multiple needed labels, so that it can be called in a variety of situations below.'''
        df = wbdata.get_dataframe(variable_labels)
        df = wbdata.get_dataframe(variable_labels, country=country)
        df.reset_index(inplace=True)
        df['date'] = df['date'].astype(int)
        df.set_index(['date'], inplace=True)
        df_with_year = df.query('date==@year')
        return df_with_year
    
    
    variable_labels = {}
    
    # Edge case: not working with age bins, just total population or total male/female population (which have different labels than age bins)
    if lower_age==0 and upper_age>=80:
        if sex=="All":
            variable_labels["SP.POP.TOTL"] = "Total Population"
        elif sex=="Male":
            variable_labels["SP.POP.TOTL.MA.IN"] = "Total Male Population"
        elif sex=="Female":
            variable_labels["SP.POP.TOTL.FE.IN"] = "Total Female Population"
        return build_year_labelled_df(variable_labels, year=year, country=country)[list(variable_labels.values())[0]].iloc[0]
    
    
    # Everything below works with age bins 
    
    # First round up/round down the age boundaries so that all necessary bins will be included
    # (The fraction will be multiplied to get a corresponding smaller ratio of an age-bracket population)
    lower_age_rounded = lower_age
    lower_fraction = 0
    if lower_age%5 != 0:
        if lower_age%5 == 4:
            lower_age_rounded = lower_age - 4
            lower_fraction = (1/5)
        elif lower_age%5 == 3:
            lower_age_rounded = lower_age - 3
            lower_fraction = (2/5)
        elif lower_age%5 == 2:
            lower_age_rounded = lower_age - 2
            lower_fraction = (3/5)
        elif lower_age%5 == 1:
            lower_age_rounded = lower_age - 1
            lower_fraction = (4/5)
    upper_age_rounded = upper_age
    upper_fraction = 0
    if upper_age%5 != 4:
        if upper_age%5 == 3:
            upper_age_rounded = upper_age + 1
            upper_fraction = (4/5)
        elif upper_age%5 == 2:
            upper_age_rounded = upper_age + 2
            upper_fraction = (3/5)
        elif upper_age%5 == 1:
            upper_age_rounded = upper_age + 3
            upper_fraction = (2/5)
        elif upper_age%5 == 0:
            upper_age_rounded = upper_age + 4
            upper_fraction = (1/5)
    
    
    # Get the labels ONLY for the bins that YOU NEED, using a loop that adds to a dictionary as you go
    placehold = lower_age_rounded
    
    def digit_adjust(placehold):
        '''Function that will prepare ages to be used to make variable_labels; turns an int to a string, and reformats single-digit numbers.'''
        if placehold < 10:
            return str(0)+str(placehold)
        else:
            return str(placehold)
    
    while placehold <= upper_age_rounded:
        while placehold < 80:
            if sex=="All":
                variable_labels["SP.POP."+digit_adjust(placehold)+digit_adjust(placehold+4)+".MA"] = str(placehold)+"-"+str(placehold+4)+" Male"
                variable_labels["SP.POP."+digit_adjust(placehold)+digit_adjust(placehold+4)+".FE"] = str(placehold)+"-"+str(placehold+4)+" Female"
            elif sex=="Male":
                variable_labels["SP.POP."+digit_adjust(placehold)+digit_adjust(placehold+4)+".MA"] = str(placehold)+"-"+str(placehold+4)+" "+sex
            elif sex=="Female":
                variable_labels["SP.POP."+digit_adjust(placehold)+digit_adjust(placehold+4)+".FE"] = str(placehold)+"-"+str(placehold+4)+" "+sex
            placehold += 5
            
            if placehold >= 80:
                if sex=="All":
                    variable_labels["SP.POP.80UP.MA"] = "80+ Male"
                    variable_labels["SP.POP.80UP.FE"] = "80+ Female"
                elif sex=="Male":
                    variable_labels["SP.POP.80UP.MA"] = "80+ Male"
                elif sex=="Female":
                    variable_labels["SP.POP.80UP.FE"] = "80+ Female"
            break
            
            
    # Now you have a 'variable_labels' that is filled with every label you need; compute final calculations:
    final_df = build_year_labelled_df(variable_labels, year=year, country=country)
    bin_count = len(list(variable_labels.keys()))
    added = 0
    i = 0
    
    
    # Edge case: if you're working with LESS THAN ONE AGE BIN
    # (1) For when the ages are different, but smaller than a single age bin (ex: 3-4 year olds)
    age_difference = upper_age - lower_age
    if age_difference<4 and bin_count==2 and sex=='All': 
        fraction = (age_difference + 1)/5
        while i < bin_count:
            added += (fraction*final_df.iloc[:, i].tolist()[0])
            i+=1
    if age_difference<4 and bin_count==1 and (sex=='Male' or sex=='Female'): 
        fraction = (age_difference + 1)/5
        return np.round(fraction * final_df.iloc[:, 0].tolist()[0])
    
    # (2) For when you're working with a single age (ex: 3-year-olds)
    if lower_age == upper_age:
        fraction = (1/5)
        if sex=="All":
            while i < 2:
                added += (fraction*final_df.iloc[:, i].tolist()[0])
                i+=1
        else:
            return np.round(fraction * final_df.sum(axis=1).tolist()[0])
    
    
    
    # FINAL OUTPUTS FOR 1 OR MORE AGE BINS: (4 different variations of cases)
    
    # CASE 1: If no rounding was needed (perfect bins):
    if lower_age%5==0 and upper_age%5==4:
        return final_df.sum(axis=1).tolist()[0]
        
        
    # CASE 2: If lower_age was rounded and upper_age wasn't:
    elif lower_age%5!=0 and upper_age%5==4:
        # For sex="All", TWO COLUMNS will need to be adjusted (Male AND Female)
        if sex=="All":
            while i < bin_count:
                while i < 2:
                    added += (lower_fraction*final_df.iloc[:, i].tolist()[0])
                    i+=1
                added += final_df.iloc[:, i].tolist()[0]
                i+=1
        else:
            while i < bin_count:
                while i < 1:
                    added += (lower_fraction*final_df.iloc[:, i].tolist()[0])
                    i+=1
                added += final_df.iloc[:, i].tolist()[0]
                i+=1
            
    
    # CASE 3: If upper_age was rounded and lower_age wasn't:
    elif lower_age%5==0 and upper_age%5!=4:
        if sex=="All":
            while i < bin_count:
                while i < (bin_count - 2):
                    added += final_df.iloc[:, i].tolist()[0]
                    i+=1
                added += (upper_fraction*final_df.iloc[:, i].tolist()[0])
                i+=1
            return np.round(added)
        else:
            while i < bin_count:
                while i < (bin_count - 1):
                    added += final_df.iloc[:, i].tolist()[0]
                    i+=1
                added += (upper_fraction*final_df.iloc[:, i].tolist()[0])
                i+=1
        
            
    # CASE 4: If both upper_age and lower_age were rounded:
    else:
        if sex=="All":
            while i < bin_count:
                while i < 2:
                    added += (lower_fraction*final_df.iloc[:, i].tolist()[0])
                    i+=1
                while i>1 and i<(bin_count - 2):
                    added += final_df.iloc[:, i].tolist()[0]
                    i+=1
                added += (upper_fraction*final_df.iloc[:, i].tolist()[0])
                i+=1
        else:
            while i < bin_count:
                while i < 1:
                    added += (lower_fraction*final_df.iloc[:, i].tolist()[0])
                    i+=1
                while i>0 and i<(bin_count - 1):
                    added += final_df.iloc[:, i].tolist()[0]
                    i+=1
                added += (upper_fraction*final_df.iloc[:, i].tolist()[0])
                i+=1 
                
                
    return np.round(added)
                

#### Test cases for population statistic function:

In [11]:
population("WLD", 2000, "Male", 0, 100) > 7e9

False

In [8]:
population("USA", 1993, "Female", 66, 73, ) == 8269864.0
population("USA", 1993, "All", 3, 11) == 34293840.0
population("USA", 1993, "All", 3, 3) == 3976670.0
population("USA", 1993, "Female", 3, 4) == 3886545.0
population("USA", 1993, "All", 3, 4) == 7953340.0
population("USA", 1993, "All", 5, 6) == 7621651.0

True

## Population Dataframes (Daysha, Hannah, William)

(Describe here)

In [ ]:
# code for population dataframes

## Hi Hannah (Title)

Start to describe your rural/urban stuff here

In [11]:
# Give variable for clarity
variable_labels = {'SP.URB.GROW' : 'upg', 'SP.RUR.TOTL.ZG': 'rpg'}

# world = wbdata.get_dataframe(variable_labels, country="WLD")
prk_pop_grow = wbdata.get_dataframe(variable_labels, country='PRK')
prk_pop_grow
# Date index is of type string; change to integers
# nzl_gdp.index = world.index.astype(int)
# usa_gdp.index = world.index.astype(int)


,upg,rpg
date,,
2020,0.834712,-0.216466
2019,0.834074,-0.163550
2018,0.827629,-0.108409
2017,0.817680,-0.057201
2016,0.806606,-0.011742
...,...,...
1964,4.043233,0.033564
1963,3.917962,-0.089348
1962,4.090284,0.082355


In [13]:
fig = px.line(prk_pop_grow, x=prk_pop_grow.index, y=['upg', 'rpg'])
fig['layout']['xaxis']['autorange'] = "reversed"
fig.show()

Growth rates are almost the same between 1971-1979. What might be the cause?
Mirroring each other in the opposite directions between 1980 and 2000.
Why do they both fall between 2016 and 2020?
However, this visualization might be misleading because as we see below, the urban population remains relatively steady at around 50 M. 

In [14]:
# Give variable for clarity
variable_labels = {'SP.URB.TOTL' : 'urban pop', 'SP.RUR.TOTL': 'rural pop'}

world_pop_urban_rural = wbdata.get_dataframe(variable_labels, country="WLD")
prk_pop_urban_rural = wbdata.get_dataframe(variable_labels, country='PRK')
# Date index is of type string; change to integers
# nzl_gdp.index = world.index.astype(int)
# usa_gdp.index = world.index.astype(int)

In [15]:
fig = px.line(prk_pop_urban_rural, x=prk_pop_urban_rural.index, y=['urban pop', 'rural pop'])
fig['layout']['xaxis']['autorange'] = "reversed"
fig.show()